In [ ]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import fiat_toolbox.well_being.methods as em

The well_being.methods module provides set of functions that are used in the well-being calculations.

The utility derived from consumption is calculated using a constant relative risk aversion (CRRA) utility function, with η
the elasticity of the marginal utility of consumption.

### CRRA Utility Function

- When ($\eta \neq 1 $):

  $$
  U(C) = \frac{C^{1-\eta}}{1 - \eta}
  $$

- When ( $\eta = 1$) (logarithmic utility):

  $$
  U(C) = \ln(C)
  $$


**Higher η values (𝜂>1):**

Stronger diminishing returns to consumption.
A higher preference for equality—well-being is more sensitive to reductions in consumption than to increases.
More weight is given to changes in consumption at lower levels.

**Lower η values (η<1):**

Weaker diminishing returns.
Well-being is less sensitive to inequality.
Changes in consumption at high levels have more impact.

Utility values themselves do not have an absolute interpretation; instead, they are useful for comparisons between different scenarios.

Here we assume that the income is the only source of consumption and the utility is calculated as a function of income.

For a specific income range we test different η values and calculate the utility curves. We normalize the utility curves to allow for comparison.

In [ ]:
# Create a range of eta values and income values
eta_values = np.round(np.arange(0.2, 2, 0.2), decimals=1)
income = np.linspace(10000, 200000, 500)
# Plot each line with a different eta value
for eta in eta_values:
    utility_values = em.utility(consumption=income, eta=eta, normalize=True)
    sb.lineplot(x=income, y=utility_values, label=f'η={eta}')
# Add labels and title
plt.xlabel('Income')
plt.ylabel('Utility')
plt.title('Utility vs Income for Different Eta Values')
plt.legend()
# Show the plot
plt.show()

Assuming that the reconstruction process follows an exponential decay model, and that a household has recovered when 95% of the reconstruction is completed, we can estimate the time it takes for a household to recover. The time to recovery is calculated as follows:

$$
\tau = \ln\left(\frac{1}{1-0.95}\right) \lambda^{-1}
$$

The λ will describe the % that is recovered at each time step. So e.g., if λ = 0.05/year, it means that the household recovers 5% of the remaining reconstruction costs each year. Then the recovery time will be in years. 

In [ ]:
rates = np.linspace(0.3, 10, 100)
# Plot each line with a different eta value
recovery_times = em.recovery_time(rates)
sb.lineplot(x=rates, y=recovery_times, label='rebuilt_per=95%')
# Add labels and title
plt.xlabel('Recovery Rate')
plt.ylabel('Recovery Time')
plt.title('Recovery Time vs Recovery Rate')
plt.legend()

# Show the plot
plt.show()

Let's assume that we would like to calculate the well-being losses for a household in an arbitrary area over a recovery period of 10 years after a flood took place that damaged the building. Let's assume that the value of the building structure (**k_str**) is 30,000 USD and that the relative damage caused by the flood (**v**) was 70%.

In [ ]:
t_max = 10 # recovery time considered in years
year_n = 52 # number of points we want to evaluate in a year
times = np.linspace(0, t_max, t_max * year_n) # time points
lambdas = np.array([0.3, 0.4, 0.5]) # recovery rates to test
v = 0.7 # the loss ratio which is repair cost divided by the total building structure value
k_str = 30000 # building structure value

We can then calculate the reconstruction costs over time the '**reconstruction_cost_t**' function following the formula:
$$
c_{reco}(t) = \lambda \nu k^{str} e^{-\lambda t}
$$

The recovery rate describes how quickly the household will repair the building. The highest the value the quickest the household will recover. Here we test 3 different values of λ: 0.3, 0.4, and 0.5.

In [ ]:
lambdas = np.array([0.3, 0.4, 0.5]) # recovery rates to test

In [ ]:
# Calculate repair costs for all combinations
repair_costs = em.ReconstructionCost(t=times, l=lambdas, v=v, k_str=k_str)
# Plot the repair costs
for  i, _lambda in enumerate(lambdas):
    sb.lineplot(x=times, y=repair_costs.losses_t[:, i], label=f"lambda={_lambda}")
# Add labels and title
plt.xlabel('Time (years)')
plt.ylabel('Repair Costs ($/year)')
plt.title('Repair Costs vs Time')
plt.legend()

Besides the reconstruction costs a household will suffer income losses relative to the loss of capital as well. The capital in this cases is the value of the building and we use a the productivity of capital index π to calculate the income losses. The income losses are calculated as follows:
$$
\Delta i (t) = \pi \nu k^{str} e^{-\lambda t}
$$


In [ ]:
pi = 0.15 # productivity of capital index
# Calculate income for all combinations
income_loss = em.IncomeLoss(t=times, l=lambdas, v=v, k_str=k_str, pi=pi)
# Plot the income loss
for  i, _lambda in enumerate(lambdas):
    sb.lineplot(x=times, y=income_loss.losses_t[:, i], label=f"lambda={_lambda}")
# Add labels and title
plt.xlabel('Time (years)')
plt.ylabel('Income Loss ($/year)')
plt.title('Income loss vs Time')
plt.legend()

Then we can calculate the consumption losses over time by simply adding up the reconstruction costs and the income losses:

$$
\Delta c (t) = c_{reco}(t) + \Delta i (t)
$$

In [ ]:
consumption_loss = em.ConsumptionLoss(t=times, l=lambdas, v=v, k_str=k_str, pi=pi)
# Plot the consumption loss
for  i, _lambda in enumerate(lambdas):
    sb.lineplot(x=times, y=consumption_loss.losses_t[:, i], label=f"lambda={_lambda}")
# Add labels and title
plt.xlabel('Time (years)')
plt.ylabel('Consumption loss ($/year)')
plt.title('Consumption loss vs Time')
plt.legend()

Assuming that the household pre-disaster consumption is described by it's yearly income (c0), we can calculate the utility losses over time as well, using the utility function as described above and assuming an elasticity of the marginal utility of consumption of η = 1.5.

In [ ]:
c0 = 48000 # yearly income
eta = 1.5 # elasticity of the marginal utility

# Calculate utility for all combinations
dut = em.UtilityLoss(t=times, l=lambdas, v=v, k_str=k_str, pi=pi, c0=c0, eta=eta)
# Plot the utility loss
for i, _lambda in enumerate(lambdas):
    valid_indices = ~np.isnan(dut.losses_t[:, i])
    sb.lineplot(x=times[valid_indices], y=dut.losses_t[valid_indices, i], label=f"lambda={_lambda}")

# Add labels and title
plt.xlabel('Time')
plt.ylabel('Utility loss')
plt.title('Utility loss vs Time')
plt.legend()
plt.show()

To calculate the optimal recovery rate for a household we can assume that the household will try to minimize the utility losses over the recovery period.

In [ ]:
c_avg = 48000 # average income for the area of interest

# Define a range of lambda values to test
t_max = 10 # recovery time considered in years
year_n = 520 # how many points we want to evaluate in a year
times = np.linspace(0, t_max, t_max * year_n)
l_min = em.recovery_rate(t_max)
_lambdas_test = np.geomspace(l_min, 10, 1000)
dut = em.UtilityLoss(times, _lambdas_test, v=v, k_str=k_str, pi=pi, c0=c0, eta=eta)
du = dut.total(rho=0, method="discrete")
# Find the lambda value corresponding to the minimum utility loss
min_dw_index = np.nanargmin(du)
min_lambda = _lambdas_test[min_dw_index]

# plot results
plt.plot(_lambdas_test, du)
plt.axvline(x=min_lambda, color='r', linestyle='--', label=f'Min Utility Loss at λ={min_lambda:.2f}')
plt.xscale('log')
plt.xlabel('λ')
plt.ylabel('Utility loss (-)')
plt.legend()
plt.show()

In [ ]:
c_avg = 48000 # average income for the area of interest

# Define a range of lambda values to test
t_max = 10 # recovery time considered in years
year_n = 52 # how many points we want to evaluate in a year
times = np.linspace(0, t_max, t_max * year_n)
l_min = em.recovery_rate(t_max)
_lambdas_test = np.geomspace(l_min, 10, 1000)
dut = em.UtilityLoss(times, _lambdas_test, v=v, k_str=k_str, pi=pi, c0=c0, eta=eta)
du = dut.total(rho=0, method="scipy")
# Find the lambda value corresponding to the minimum utility loss
min_dw_index = np.nanargmin(du)
min_lambda = _lambdas_test[min_dw_index]

# plot results
plt.plot(_lambdas_test, du)
plt.axvline(x=min_lambda, color='r', linestyle='--', label=f'Min Utility Loss at λ={min_lambda:.2f}')
plt.xscale('log')
plt.xlabel('λ')
plt.ylabel('Utility loss (-)')
plt.legend()
plt.show()

In [ ]:
import time
start_time = time.time()
optimal_lambda = em.opt_lambda(v, k_str, c0, pi, eta, times=times, lmin=0.3, lmax=10, method="discrete")
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")
print(f"Optimal lambda: {optimal_lambda}")

In [ ]:
start_time = time.time()
optimal_lambda = em.opt_lambda(v, k_str, c0, pi, eta, t_max=t_max, lmin=0.3, lmax=10, method="scipy")
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")
print(f"Optimal lambda: {optimal_lambda}")